In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests
import os
import numpy as np

## 1. Thu thập đường dẫn của dữ liệu

In [100]:
def Collect_Href_of_Diamonds(shape):
    url = "https://www.allurez.com/loose-diamonds/?filterVal=yes&shape={}".format(shape)

    # Khởi tạo trình duyệt Chrome
    ser = Service("C:/Program Files (x86)/chromedriver.exe")
    driver = webdriver.Chrome(service = ser)

    # Mở trang web
    driver.get(url)
    driver.fullscreen_window()

    # Thay đổi view
    time.sleep(5)
    link = driver.find_element(By.ID, 'grid_view')
    link.click()

    link_href = set()

    # Nhấn Seemore
    while True:
        try:
            time.sleep(5)
            # Find the link by its attributes using the By.XPATH locator
            see_more_button = driver.find_element(By.XPATH, '//a[@class="btn_new2" and text()="See More..."]')
            # Click the link
            see_more_button.click()
            # driver.implicitly_wait(60)
            time.sleep(5)
            # Find data
            divs = driver.find_elements(By.CLASS_NAME, "name")

            # Find data
            soup = BeautifulSoup(driver.page_source, "html.parser")
            # Find the <ul> tag with class "dmd_grid_view"
            ul_tag = soup.find("ul", {"class": "dmd_grid_view"})
            if(ul_tag != None):
                # Extract all the <li> tags inside the <ul> tag
                li_tags = ul_tag.find_all("li")
                for li in li_tags:
                    d = li.find("div", {"class" : "name"})
                    if(d != None):
                        a = d.find("a")
                        link_href.add(a.get("href"))
            
            if(len(link_href) > 1500):
                return link_href

        except NoSuchElementException:
            break
        except StaleElementReferenceException:
            break
        except ElementNotInteractableException:
            break
    # Find data
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # Find the <ul> tag with class "dmd_grid_view"
    ul_tag = soup.find("ul", {"class": "dmd_grid_view"})
    if(ul_tag != None):
        # Extract all the <li> tags inside the <ul> tag
        li_tags = ul_tag.find_all("li")
        for li in li_tags:
            d = li.find("div", {"class" : "name"})
            if(d != None):
                a = d.find("a")
                link_href.add(a.get("href"))

    driver.quit()
    return link_href

### Thu thập đường dẫn của từng Shape và đưa vào thư mục Link_Diamonds

In [93]:
shapes = ["Round", "Princess", "Cushion", "Emerald", "Asscher", "Marquise", "Oval", "Radiant", "Pear", "Heart"]

for shape in shapes:
    path = "./Link_Diamonds/" + shape + ".txt"
    with open(path, "w") as f:
        links = Collect_Href_of_Diamonds(shape = shape)
        for link in links:
            f.write(link + "\n")

### Gộp các đường dẫn vào 2 file link_big.txt và link_small.txt

In [2]:
shapes = ["Round", "Princess", "Cushion", "Emerald", "Asscher", "Marquise", "Oval", "Radiant", "Pear", "Heart"]

with open("link_big.txt", "w") as file:
    for shape in shapes:
        path = "./Link_Diamonds/" + shape + ".txt"
        with open(path, "r") as f:
            for line in f.readlines():
                file.write(line)

In [4]:
shapes = ["Round", "Princess", "Cushion", "Emerald", "Asscher", "Marquise", "Oval", "Radiant", "Pear", "Heart"]

with open("link_small.txt", "w") as file:
    for shape in shapes:
        path = "./Link_Diamonds/" + shape + ".txt"
        with open(path, "r") as f:
            lines = f.readlines()
            np.random.shuffle(lines)
            for line in lines[:110]:
                file.write(line)

## 2. Thu thập tất cả các dữ liệu

In [4]:
def Collect_Raw_Data(url):
    result = requests.get(url)
    html = BeautifulSoup(result.text, "html.parser")
    record = ""
    tables = html.find_all("table", {"class" : "dtlnt_infotable"})
    for table in tables:
        infors = table.find_all("tr")
        for infor in infors:
            tds = infor.find_all("td")
            if len(tds) == 2:
                key = ""
                if tds[0].find("a") == None:
                    key = tds[0].find("span").text
                else:
                    key = tds[0].find("a").text
                val = tds[1].text
                record += key.strip() + ":" + val.strip() + ","
    price = html.find("span", {"class" : "listprice dlp"})
    if price != None:
        p = price.text.replace("$", "").replace(",", "")
        record += "Price:" + p

    return record

### Thu thập dữ liệu từ file link_big.txt

In [ ]:
raw_data_big = []
with open("link_big.txt", "r") as f:
    for line in f.readlines():
        data = Collect_Raw_Data(line.rstrip("\n"))
        raw_data_big.append(data)

In [115]:
with open("./Raw_Data/data_big.txt", "w") as f:
    for raw in raw_data_big:
        f.write(raw + "\n")

#### ==> Thu được tất cả dữ liệu lấy được  trong file data_big.txt trong thư mục Raw_Data

### Thu thập dữ liệu từ file link_small.txt

In [ ]:
raw_data_small = []
with open("link_small.txt", "r") as f:
    for line in f.readlines():
        data = Collect_Raw_Data(line.rstrip("\n"))
        raw_data_small.append(data)

In [ ]:
with open("./Raw_Data/data_small.txt", "w") as f:
    for raw in raw_data_small:
        f.write(raw + "\n")

#### ==> Thu được tất cả dữ liệu lấy được  trong file data_small.txt trong thư mục Raw_Data

## 3. Chuyển đổi dữ liệu thành file csv

In [13]:
def Convert_line_to_CSV(line, columns):
    features = line.rstrip("\n").split(",")
    # lấy đặc trưng và giá trị
    keys = []
    vals = []
    for feature in features:
        keys.append(feature.split(":")[0])
        vals.append(feature.split(":")[1])
    # Ghi lên file
    record = ""
    for column in columns:
        if(column in keys):
            index = keys.index(column)
            if(column == "Price"):
                vals[index] = vals[index].replace(".", "")
            record += vals[index] + ","
        else:
            record += ","
    return record[:-1] + "\n"

### Chuyển đổi file data_big.txt trong Raw_Data thành file csv

In [14]:
columns = ['Style Number', 'Shape', 'Carat Weight', 'Color', 'Clarity', 'Graded By', 'Cut Grade', 'Fluorescence', 'Culet', 'Depth', 'Table', 'Girdle', 'Polish', 'Symmetry', 'Measurements', 'Price']

with open("./Raw_Data/data_big.csv", "w") as f:
    cols = ""
    for column in columns:
        cols += column + ","
    f.write(cols[:-1] + "\n")
    with open("./Raw_Data/data_big.txt", "r") as datas:
        for line in datas.readlines():
            record = Convert_line_to_CSV(line, columns)
            f.write(record)

### Chuyển đổi file data_small.txt trong Raw_Data thành file csv

In [18]:
columns = ['Style Number', 'Shape', 'Carat Weight', 'Color', 'Clarity', 'Graded By', 'Cut Grade', 'Fluorescence', 'Culet', 'Depth', 'Table', 'Girdle', 'Polish', 'Symmetry', 'Measurements', 'Price']

with open("./Raw_Data/data_small.csv", "w") as f:
    cols = ""
    for column in columns:
        cols += column + ","
    f.write(cols[:-1] + "\n")
    with open("./Raw_Data/data_small.txt", "r") as datas:
        for line in datas.readlines():
            record = Convert_line_to_CSV(line, columns)
            f.write(record)